# TODOS LOS FORMATOS

In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

### Ventas y Volumen

In [161]:
cols = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Sector', 'Seccion', 'Grupo de Familia', 'Ventas c/impuesto', 'Venta en Unidades']
df_ventas_vol = pd.read_csv('data/ventas y vol total CIA Briefing 2da semana total CIA.csv', encoding='utf-16', header=1, usecols=cols, decimal=',')
df_ventas_vol.columns = df_ventas_vol.columns.str.lower().str.replace(' ','_')

In [162]:
df_ventas_vol.rename(columns={
    'mes':'fecha',
    'ventas_c/impuesto':'vct',
    'venta_en_unidades':'vol',
}, inplace=True)

In [163]:
df_ventas_vol['mes'] = df_ventas_vol['fecha'].str.split(' ').str[0]
df_ventas_vol['numero_operacional'] = df_ventas_vol['punto_operacional'].str.split(' ').str[0].astype(int)

### Separacion Ventas y Volumen

In [164]:
df_ventas = df_ventas_vol[['año', 'fecha', 'direccion', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'vct', 'mes', 'numero_operacional']]
df_volumen = df_ventas_vol[['año', 'fecha', 'direccion', 'punto_operacional', 'sector', 'seccion', 'grupo_de_familia', 'vol', 'mes', 'numero_operacional']]
df_ventas.rename(columns={
    'vct':'valores'
}, inplace=True)
df_volumen.rename(columns={
    'vol':'valores'
}, inplace=True)

C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_11544\3609712812.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ventas.rename(columns={
C:\Users\Juan Mera\AppData\Local\Temp\ipykernel_11544\3609712812.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_volumen.rename(columns={


In [165]:
df_ventas['categoria'] = 'vct'
df_volumen['categoria'] = 'vol'

In [166]:
df_ventas['valores'] = pd.to_numeric(df_ventas['valores'].str.replace('.', '').str.replace(',','.'))
df_volumen['valores'] = pd.to_numeric(df_volumen['valores'].str.replace('.', '').str.replace(',','.'))

### Le quito los envases al volumen

In [167]:
df_volumen = df_volumen[~df_volumen['grupo_de_familia'].isin(['ENVASES BEBIDAS', 'ENVASES PAGADOS'])]

In [168]:
df_ventas_tienda = df_ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()
df_volumen_tienda = df_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria'])['valores'].sum().reset_index()

### Padron

In [169]:
cols = ['N°', 'NOMBRE', 'Fecha apertura', 'BANDERA', 'ORGANIZACIÓN ', 'PROVINCIA', 'FIN DE CIERRE', 'ENE.2', 'FEB.2', 'MAR.2', 'ABR.2', 'MAY.2', 'JUN.2', 'JUL.2', 'AGO.2', 'SEP.2', 'OCT.2', 'NOV.2', 'DIC.2']
padron = pd.read_excel('data/padron.xlsx', header=17, usecols=cols)
padron.columns = padron.columns.str.lower().str.strip().str.replace(' ', '_').str.replace('.2', '')
padron.rename(columns={
    'n°':'numero_operacional',
}, inplace=True)

In [170]:
#Elimino las filas que tengan na en su numero, nombre y mes comparable
padron = padron.dropna(subset=['numero_operacional', 'nombre', 'sep'], how='any')

In [171]:
padron['numero_operacional'] = padron['numero_operacional'].astype(int)

### Debitos total Tienda

In [172]:
df_debitos_tienda = pd.read_csv('data/debitos x tienda total CIA Briefing 2da semana total CIA.csv', encoding='utf-16', header=1, decimal=',')
df_debitos_tienda.columns = df_debitos_tienda.columns.str.lower().str.replace(' ', '_')
df_debitos_tienda.rename(columns={
    'cant._tickets_por_local':'valores',
    'mes':'fecha',
}, inplace=True)
df_debitos_tienda['valores'] = pd.to_numeric(df_debitos_tienda['valores'].str.replace('.', '').str.replace(',','.'))
df_debitos_tienda['mes'] = df_debitos_tienda['fecha'].str.split(' ').str[0]
df_debitos_tienda['numero_operacional'] = df_debitos_tienda['punto_operacional'].str.split(' ').str[0].astype(int)
df_debitos_tienda = df_debitos_tienda.drop(columns=['indicadores', 'fecha'])
df_debitos_tienda['categoria'] = 'deb'
df_debitos_tienda = df_debitos_tienda[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'valores']]

### Concateno todo nivel tienda y Join con el Padron

In [173]:
df_tienda = pd.concat([df_ventas_tienda, df_debitos_tienda, df_volumen_tienda])
df_tienda['numero_operacional'] = df_tienda['numero_operacional'].astype(int)

In [174]:
df_tienda_join = pd.merge(df_tienda, padron, how='left', on='numero_operacional')
df_tienda_join = df_tienda_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'provincia', 'fecha_apertura', 'fin_de_cierre','categoria', 'sep', 'valores']]
df_tienda_comparable = df_tienda_join[df_tienda_join['sep'] == 'SC']

#Este df NO COMPARABLE me sirve para calcular las progresiones 28 dias moviles por formato
df_tienda_no_comparable = df_tienda_join

### Genero un df por Superficie comparable, pivoteo la info y calculo las porgresiones totales por tienda por categoria

In [175]:
df_tienda_comparable = df_tienda_comparable.pivot_table(values='valores', index=['direccion', 'numero_operacional', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()
df_tienda_comparable['progresion'] = round((df_tienda_comparable[2025] / df_tienda_comparable[2024]) - 1, 3)
df_tienda_comparable = df_tienda_comparable.sort_values(by='progresion', ascending=False)

#Me sirve para luego la consolidacion de informacion
df_tienda_comparable_aux = df_tienda_comparable

In [176]:
df_tienda_no_comparable = df_tienda_no_comparable.pivot_table(values='valores', index=['direccion', 'numero_operacional', 'punto_operacional', 'categoria'], columns='año', aggfunc='sum').reset_index()
df_tienda_no_comparable['progresion'] = round((df_tienda_no_comparable[2025] / df_tienda_no_comparable[2024]) - 1, 3)
df_tienda_no_comparable = df_tienda_no_comparable.sort_values(by='progresion', ascending=False)

### Genero un DF por superficie comparable, agrupo la informacion a nivel formato para calular sus progresiones por categoria

In [177]:
df_formato_comparable = df_tienda_comparable.groupby(['direccion', 'categoria'])[[2024, 2025]].sum().reset_index()
df_formato_comparable['progresion'] = round(df_formato_comparable[2025] / df_formato_comparable[2024] - 1, 3)
df_formato_comparable_final = df_formato_comparable.sort_values(['categoria'])

In [178]:
df_formato_no_comparable = df_tienda_no_comparable.groupby(['direccion', 'categoria'])[[2024, 2025]].sum().reset_index()
df_formato_no_comparable['progresion'] = round(df_formato_no_comparable[2025] / df_formato_no_comparable[2024] - 1, 3)
df_formato_no_comparable_final = df_formato_no_comparable.sort_values(['categoria'])

### Debitos por Sector

In [179]:
df_debitos_sector = pd.read_csv('data/debitos x sector total CIA Briefing 2da semana total CIA.csv', encoding='utf-16', header=1, decimal=',')

In [180]:
df_debitos_sector.columns = df_debitos_sector.columns.str.strip().str.lower().str.replace(' ', '_')
df_debitos_sector = df_debitos_sector.rename(columns={
    'cantidad_de_tickets':'valores',
    'mes':'fecha'
})
df_debitos_sector = df_debitos_sector.drop(columns=['indicadores'])
df_debitos_sector['mes'] = df_debitos_sector['fecha'].str.split(' ').str[0]
df_debitos_sector['numero_operacional'] = df_debitos_sector['punto_operacional'].str.split(' ').str[0]
df_debitos_sector['categoria'] = 'deb'
df_debitos_sector['valores'] = pd.to_numeric(df_debitos_sector['valores'].str.replace('.', '').str.replace(',','.'))

### Agrupo las tres categorias por Sector, las joineo con el padron, me quedo con los SC, calculo proresiones y pivoteo para mostrar mejor la informacion

In [181]:
df_ventas_sector = df_ventas.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector'])['valores'].sum().reset_index()
df_volumen_sector = df_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector'])['valores'].sum().reset_index()
df_debitos_sector = df_debitos_sector[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 'valores']]

In [182]:
df_sector = pd.concat([df_ventas_sector, df_volumen_sector, df_debitos_sector])
df_sector['numero_operacional'] = df_sector['numero_operacional'].astype(int) 
df_sector_join = pd.merge(df_sector, padron, on='numero_operacional', how='left')
df_sector_join = df_sector_join[['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'provincia', 'fecha_apertura', 'fin_de_cierre', 'categoria', 'sector', 'sep', 'valores']]

In [183]:
df_sector_comparable = df_sector_join[df_sector_join['sep'] == 'SC']

In [184]:
df_sector_comparable = df_sector_comparable.pivot_table(values='valores', index=['direccion' ,'numero_operacional', 'punto_operacional', 'categoria', 'sector'], columns='año', aggfunc='sum').reset_index()
df_sector_comparable['progresion'] = round((df_sector_comparable[2025] / df_sector_comparable[2024]) - 1, 3)

In [185]:
#Sirve para calcular las progresiones por sector a nivel Formato
df_formato_sector_comparable = df_sector_comparable

#Sirve Para realizar una baja consolidada de informacion
df_formato_sector_comparable_aux = df_formato_sector_comparable

In [186]:
df_progresiones_categoria_sectores = df_sector_comparable.pivot_table(values='progresion', index=['numero_operacional', 'punto_operacional', 'categoria'], columns='sector', aggfunc='sum').reset_index()

In [187]:
df_tienda_comparable = df_tienda_comparable.rename(columns={'progresion':'total_tienda'})

### Trabajo con la Info para Obtener las progresiones por Sector a Nivel Formato

In [188]:
df_formato_sector_comparable = df_formato_sector_comparable.groupby(['direccion', 'categoria', 'sector'])[[2024, 2025]].sum().reset_index()

In [189]:
df_formato_sector_comparable['progresion'] = round(df_formato_sector_comparable[2025] / df_formato_sector_comparable[2024] - 1, 3)

In [190]:
df_formato_sector_comparable = df_formato_sector_comparable.pivot_table(values='progresion', index=['direccion', 'categoria'], columns='sector', aggfunc='sum').reset_index()
df_formato_sector_comparable = df_formato_sector_comparable.fillna(0)

### Concateno las progresiones por sectores con el primer df utilziando su numero operacional. Luego lo limpio y lo presento

In [191]:
df_progresiones_join_sector_tienda = pd.merge(df_progresiones_categoria_sectores, df_tienda_comparable[['direccion', 'numero_operacional', 'categoria', 'total_tienda']], on=['numero_operacional', 'categoria'], how='left')

In [192]:
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.fillna(0)

In [193]:
df_progresiones_join_sector_tienda.columns = df_progresiones_join_sector_tienda.columns.str.capitalize().str.strip().str.replace('_', ' ')

In [194]:
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.drop(columns=['Numero operacional'])
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.rename(columns={'Total tienda': 'Total tienda', 'P.g.c.': 'PGC'})
df_progresiones_join_sector_tienda = df_progresiones_join_sector_tienda.sort_values(by='Total tienda', ascending=False)

In [195]:
df_final_consolidado_tienda = df_tienda_comparable.drop(columns=[2024, 2025])
df_final_consolidado_tienda = df_final_consolidado_tienda.rename(columns={'total_tienda':'progresion'})
df_final_consolidado_tienda['sector'] = 'Total'

In [196]:
df_final_consolidado_sector = df_sector_comparable.drop(columns=[2024, 2025])
df_final_consolidado_sector[['direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'progresion', 'sector']]

año,direccion,numero_operacional,punto_operacional,categoria,progresion,sector
0,E-COMMERCE,899,899 - E-Commerce,deb,-0.747,BAZAR
1,E-COMMERCE,899,899 - E-Commerce,deb,-0.293,ELECTRODOMESTICOS
2,E-COMMERCE,899,899 - E-Commerce,deb,1.000,P.G.C.
3,E-COMMERCE,899,899 - E-Commerce,deb,NaN,TEXTIL
4,E-COMMERCE,899,899 - E-Commerce,vct,-0.734,BAZAR
...,...,...,...,...,...,...
11610,PROXIMIDAD,874,874 - Mini Caraffa Claudio Cuenca 1784 Cb,vol,-0.750,ELECTRODOMESTICOS
11611,PROXIMIDAD,874,874 - Mini Caraffa Claudio Cuenca 1784 Cb,vol,0.319,OTROS
11612,PROXIMIDAD,874,874 - Mini Caraffa Claudio Cuenca 1784 Cb,vol,0.317,P.G.C.
11613,PROXIMIDAD,874,874 - Mini Caraffa Claudio Cuenca 1784 Cb,vol,0.509,PRODUCTOS FRESCO


In [197]:
df_final_consolidado_total = pd.concat([df_final_consolidado_sector, df_final_consolidado_tienda])
df_final_consolidado_total = df_final_consolidado_total.pivot_table(values='progresion', index=['direccion', 'punto_operacional'], columns=['categoria', 'sector'], aggfunc='sum').reset_index()

### Trabajo sobre el ultimo punto "Grupo de Familia". Agrupo, joineo con el padron, me quedo con los SC, calculo el gap y la CMG, limpio la info y presento

In [198]:
df_volumen_grupo_de_familia = df_volumen.groupby(['año', 'mes', 'direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 'seccion', 'grupo_de_familia'])['valores'].sum().reset_index()

In [199]:
df_volumen_grupo_de_familia_join = pd.merge(df_volumen_grupo_de_familia, padron[['numero_operacional', 'sep']], on='numero_operacional', how='left')

In [200]:
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_join[df_volumen_grupo_de_familia_join['sep'] == 'SC']

In [201]:
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.pivot_table(values='valores', index=['direccion', 'grupo_de_familia', 'seccion', 'categoria'], columns='año', aggfunc='sum').reset_index()

In [202]:
df_volumen_grupo_de_familia_comparable[2024] = df_volumen_grupo_de_familia_comparable[2024].fillna(0)

In [203]:
df_volumen_grupo_de_familia_comparable['GAP'] = df_volumen_grupo_de_familia_comparable[2025] - df_volumen_grupo_de_familia_comparable[2024]

In [204]:
df_volumen_grupo_de_familia_comparable['progresion'] = (df_volumen_grupo_de_familia_comparable[2025] / df_volumen_grupo_de_familia_comparable[2024]) - 1

In [205]:
df_volumen_grupo_de_familia_comparable['progresion'] = df_volumen_grupo_de_familia_comparable['progresion'].replace(np.inf, 0)
df_volumen_grupo_de_familia_comparable['progresion'] = df_volumen_grupo_de_familia_comparable['progresion'].replace(np.nan, 0)
df_volumen_grupo_de_familia_comparable.sort_values('progresion', ascending=False)

año,direccion,grupo_de_familia,seccion,categoria,2024,2025,GAP,progresion
132,HIPERMERCADO,FOTOGRAFIA,FOTOGRAFIA,vol,1.00,330.0,329.00,329.000000
410,MARKET,MERCADO DEL MAR CONGELADO,PESCADERIA,vol,5.47,1164.7,1159.23,211.925046
291,MARKET,BAGAJE,LIBRERIA,vol,6.00,405.0,399.00,66.500000
304,MARKET,CALZADO AGUA VERANO,CALZADO,vol,1.00,63.0,62.00,62.000000
121,HIPERMERCADO,ESTACIONAL NAVIDAD,SECO,vol,277.00,12839.0,12562.00,45.350181
...,...,...,...,...,...,...,...,...
429,MARKET,PLANTINES,FRUTAS Y VERDURAS,vol,1572.00,13.0,-1559.00,-0.991730
842,PROXIMIDAD,ROPA DE BANO PERMANENTE,BLANCO,vol,203.00,1.0,-202.00,-0.995074
265,HIPERMERCADO,VIDEOJUEGOS,IMAGEN,vol,7.00,0.0,-7.00,-1.000000
463,MARKET,TARJETA GIFT CARD,OTRAS VENTAS,vol,-1.00,0.0,1.00,-1.000000


In [206]:
df_volumen_grupo_de_familia_comparable['total_2024_direccion'] = df_volumen_grupo_de_familia_comparable.groupby('direccion')[2024].transform('sum')
df_volumen_grupo_de_familia_comparable['Cmg'] = df_volumen_grupo_de_familia_comparable['GAP'] / df_volumen_grupo_de_familia_comparable['total_2024_direccion']

In [207]:
df_volumen_grupo_de_familia_comparable['Cmg'] = df_volumen_grupo_de_familia_comparable['Cmg'].fillna(0)
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.sort_values('Cmg', ascending=False)
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.rename(columns={'direccion':'Direccion','grupo_de_familia':'Grupo de familia', 'seccion':'Seccion', 'categoria':'Categoria', 'progresion':'Progresion'})
df_volumen_grupo_de_familia_comparable = df_volumen_grupo_de_familia_comparable.drop(columns=['total_2024_direccion'])

### Trabajo sobre una bajada de informacion consolidada para realizar un giratorio

In [208]:
# Genero un DF auxiliar para realizar una bajada consolidada de informacion para generar un giratorio
df_tienda_comparable_aux['sector'] = ''
df_tienda_comparable_aux = df_tienda_comparable_aux[['direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 2024, 2025, 'progresion']]
df_tienda_comparable_aux['aux'] = 'tienda'

df_formato_comparable_aux = df_formato_comparable.copy()
df_formato_comparable_aux['sector'] = ''
df_formato_comparable_aux['numero_operacional'] = ''
df_formato_comparable_aux['punto_operacional'] = ''
df_formato_comparable_aux = df_formato_comparable_aux[['direccion', 'numero_operacional', 'punto_operacional', 'categoria', 'sector', 2024, 2025, 'progresion']]
df_formato_comparable_aux['aux'] = 'formato'

df_formato_sector_comparable_2_aux = df_formato_sector_comparable_aux.groupby(['direccion', 'categoria', 'sector'])[[2024, 2025]].sum().reset_index()
df_formato_sector_comparable_2_aux['progresion'] = round(df_formato_sector_comparable_2_aux[2025] / df_formato_sector_comparable_2_aux[2024] - 1, 3)
df_formato_sector_comparable_2_aux['aux'] = 'formato_sector'
df_formato_sector_comparable_2_aux['numero_operacional'] = ''
df_formato_sector_comparable_2_aux['punto_operacional'] = ''

df_formato_sector_comparable_aux['aux'] = 'tienda_sector'

df_bajada_consolidada = pd.concat([df_tienda_comparable_aux, df_formato_comparable_aux, df_formato_sector_comparable_aux, df_formato_sector_comparable_2_aux])

# GRAFICOS V2

In [209]:
deb_acum = pd.read_csv('data/debitos total CIA Acumulado Briefing 2da semana.csv', encoding='utf-16', header=1, decimal=',')
cols = ['Año', 'Mes', 'Direccion', 'Punto Operacional', 'Ventas c/impuesto']
vct_acum = pd.read_csv('data/ventas total CIA Acumulado Briefing 2da semana.csv', encoding='utf-16', header=1, usecols=cols)
vol_acum = pd.read_csv('data/volumen sin envases total CIA Acumulado Briefing 2da semana.csv', encoding='utf-16', header=1)

In [210]:
deb_acum = deb_acum.drop(columns=['Indicadores'])
vol_acum = vol_acum.drop(columns=['Indicadores'])

In [211]:
deb_acum = deb_acum.rename(columns={'Cant. Tickets por Local':'valores'})
vol_acum = vol_acum.rename(columns={'VOLUMEN':'valores'})
vct_acum = vct_acum.rename(columns={'Ventas c/impuesto':'valores'})

In [212]:
deb_acum['categoria'] = 'deb'
vol_acum['categoria'] = 'vol'
vct_acum['categoria'] = 'vct'

In [213]:
vol_acum['valores'] = pd.to_numeric(vol_acum['valores'].str.replace('.','').str.replace(',', '.'))
vct_acum['valores'] = pd.to_numeric(vct_acum['valores'].str.replace('.','').str.replace(',', '.'))
deb_acum['valores'] = pd.to_numeric(deb_acum['valores'].str.replace('.', ''))

In [214]:
acum_join = pd.concat([deb_acum, vol_acum, vct_acum])

In [215]:
acum_join.columns = acum_join.columns.str.strip().str.replace(' ', '_').str.lower()

In [216]:
acum_join['numero_operacional'] = acum_join['punto_operacional'].str.split(' ').str[0].astype(int)

In [217]:
acum_join = pd.merge(acum_join, padron[['numero_operacional', 'sep']], how='left')

In [218]:
acum_join_comparable = acum_join[acum_join['sep'] == 'SC']
acum_join_comparable = acum_join_comparable.rename(columns={'mes':'fecha'})
acum_join_comparable['mes'] = acum_join_comparable['fecha'].str.split(' ').str[0]

acum_join_no_comparable = acum_join
acum_join_no_comparable = acum_join_no_comparable.rename(columns={'mes':'fecha'})
acum_join_no_comparable['mes'] = acum_join_no_comparable['fecha'].str.split(' ').str[0] 

In [219]:
acum_join_comparable = acum_join_comparable.groupby(['año', 'fecha', 'mes', 'direccion', 'categoria'])['valores'].sum().reset_index()

acum_join_no_comparable = acum_join_no_comparable.groupby(['año', 'fecha', 'mes', 'direccion', 'categoria'])['valores'].sum().reset_index()

In [220]:
meses_orden = {'Enero':1, 'Febrero':2, 'Marzo':3, 'Abril':4, 'Mayo':5, 'Junio':6, 'Julio':7, 'Agosto':8, 'Septiembre':9, 'Octubre':10, 'Noviembre':11, 'Diciembre':12}

In [221]:
acum_join_comparable['aux'] = acum_join_comparable['mes'].map(meses_orden)

acum_join_no_comparable['aux'] = acum_join_no_comparable['mes'].map(meses_orden)

In [222]:
meses_invertidos = {v:k for k, v in meses_orden.items()}

In [223]:
acum_join_comparable.sort_values(by='aux', ascending=True)
acum_join_comparable['fecha_completa'] = pd.to_datetime(
    '01/' + acum_join_comparable['aux'].astype(str) + '/' + acum_join_comparable['año'].astype(str), format='%d/%m/%Y')
mes_comparable = 'Septiembre'
fecha_tope = pd.to_datetime('01/'+ str(meses_orden[mes_comparable]) + '/' + '2025', format='%d/%m/%Y')


acum_join_no_comparable.sort_values(by='aux', ascending=True)
acum_join_no_comparable['fecha_completa'] = pd.to_datetime(
    '01/' + acum_join_no_comparable['aux'].astype(str) + '/' + acum_join_no_comparable['año'].astype(str), format='%d/%m/%Y')
mes_comparable = 'Septiembre'
fecha_tope = pd.to_datetime('01/'+ str(meses_orden[mes_comparable]) + '/' + '2025', format='%d/%m/%Y')

In [224]:
acum_join_comparable = acum_join_comparable[acum_join_comparable['fecha_completa'] <= fecha_tope]

acum_join_no_comparable = acum_join_no_comparable[acum_join_no_comparable['fecha_completa'] <= fecha_tope]

In [225]:
acum_join_comparable = acum_join_comparable.pivot_table(values='valores', columns='año', index=['direccion', 'mes', 'categoria', 'aux'], aggfunc='sum').reset_index()

acum_join_no_comparable = acum_join_no_comparable.pivot_table(values='valores', columns='año', index=['direccion', 'mes', 'categoria', 'aux'], aggfunc='sum').reset_index()

In [226]:
acum_join_comparable['progresion 2024'] = round((acum_join_comparable[2024] / acum_join_comparable[2023]) - 1, 3)
acum_join_comparable['progresion 2025'] = round((acum_join_comparable[2025] / acum_join_comparable[2024]) - 1, 3)

acum_join_no_comparable['progresion 2024'] = round((acum_join_no_comparable[2024] / acum_join_no_comparable[2023]) - 1, 3)
acum_join_no_comparable['progresion 2025'] = round((acum_join_no_comparable[2025] / acum_join_no_comparable[2024]) - 1, 3)

In [227]:
acum_join_comparable = acum_join_comparable.sort_values(by=['direccion', 'categoria' ,'aux'], ascending=[True, True, True])
acum_join_comparable = acum_join_comparable.drop(columns=['aux'])

acum_join_no_comparable = acum_join_no_comparable.sort_values(by=['direccion', 'categoria' ,'aux'], ascending=[True, True, True])
acum_join_no_comparable = acum_join_no_comparable.drop(columns=['aux'])

### Genero dos bucles para poder realizar varios filtros en las tablas importantes y asi conseguir un archivo por formato donde se muestre la informacion Total a Nivel Formato, Tienda y Sector

In [228]:
from pathlib import Path

formatos = df_tienda_comparable['direccion'].unique().tolist()
categorias = ['vct', 'deb', 'vol']

try:
    out_dir = Path("results")
    out_dir.mkdir(parents=True, exist_ok=True)

    for formato in formatos:
        out_file = out_dir / f"Resultados Briefing {formato.upper()} ({datetime.today().strftime('%d-%m-%Y')}).xlsx"

        with pd.ExcelWriter(out_file) as writer:

            df_formato_comparable_final[df_formato_comparable_final['direccion'] == formato].to_excel(writer, sheet_name=f'Total Categoria - {formato[0:3]}', index=False)

            df_formato_no_comparable_final[df_formato_no_comparable_final['direccion'] == formato].to_excel(writer, sheet_name=f'Total Categoria (Sup Total) - {formato[0:3]}'[0:31], index=False)

            df_formato_sector_comparable[df_formato_sector_comparable['direccion'] == formato].to_excel(writer, sheet_name=f'Total Categoria x Sector - {formato[0:3]}', index=False)
            
            for categoria in categorias:

                df_filtrado = df_progresiones_join_sector_tienda[(df_progresiones_join_sector_tienda['Categoria'] == categoria) & (df_progresiones_join_sector_tienda['Direccion'] == formato)]
                df_filtrado = df_filtrado.drop(columns=['Direccion'])
                
                df_filtrado.to_excel(writer, sheet_name=f'{categoria} - {formato[0:3]}', index=False)

            df_final_consolidado_total[df_final_consolidado_total['direccion'] == formato].to_excel(writer, sheet_name=f'Info Consolidada - {formato[0:3]}', index=True)
            
            df_volumen_grupo_de_familia_comparable[df_volumen_grupo_de_familia_comparable['Direccion'] == formato].to_excel(writer, sheet_name=f'GF Consolidada - {formato[0:3]}', index=False)

            acum_join_comparable[acum_join_comparable['direccion'] == formato].to_excel(writer, sheet_name=f'Progresiones comp - {formato[0:3]}', index=False)

            acum_join_no_comparable[acum_join_no_comparable['direccion'] == formato].to_excel(writer, sheet_name=f'Progresiones total - {formato[0:3]}', index=False)

            #Por el momento la quito ya que primero tengo que pensar como realizar el giratorio
            #df_bajada_consolidada.to_excel(writer, sheet_name=f'Base Giratorio', index=False)
            
        print(f"Archivo guardado en: {out_file}")

except Exception as e:
    print(f"No se logró guardar la información en un archivo Excel. ERROR: {e}")

Archivo guardado en: results\Resultados Briefing NO INFORMADO (22-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing PROXIMIDAD (22-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing MARKET (22-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing MAXI (22-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing HIPERMERCADO (22-10-2025).xlsx
Archivo guardado en: results\Resultados Briefing E-COMMERCE (22-10-2025).xlsx
